<a href="https://colab.research.google.com/github/greyhound101/IEEE-CIS-Fraud/blob/master/autoenc_prep_without_id_without_V.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Loading Libraries

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from keras.layers import *
from keras.optimizers import *
from keras.models import *
from keras import backend as K
import keras
import random
import tensorflow as tf
import keras.backend as K

from keras.models import Model
from keras.layers import Dense, Input, Dropout, BatchNormalization, Activation
from keras.utils.generic_utils import get_custom_objects
from keras.optimizers import Adam, Nadam
from keras.callbacks import Callback
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
import gc


In [2]:

np.random.seed(42) # NumPy
random.seed(42) # Python

Loading data

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Mounted at /content/gdrive


Load dataframe and drop Vs

In [4]:

trn=pd.read_csv('/content/gdrive/My Drive/fraud/train.csv')
tst=pd.read_csv('/content/gdrive/My Drive/fraud/test.csv')
trn=trn.drop(['isFraud'],1)

Divide data as numerical and categorical and create grouped mean and std with numerical and drop id 

In [5]:

dum=list(trn.filter(regex='dum'))
isna=list(trn.filter(regex='isna'))



tot=list(set(dum+isna))
num=[ i for i in list(trn) if i not in tot]

num.remove('id') 
trn=trn.drop(['id'],1)
tst=tst.drop(['id'],1)


Reduce memory used

In [6]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df
trn=reduce_mem_usage(trn)
tst=reduce_mem_usage(tst)

Memory usage of dataframe is 2243.72 MB
Memory usage after optimization is: 551.92 MB
Decreased by 75.4%
Memory usage of dataframe is 1925.14 MB
Memory usage after optimization is: 473.55 MB
Decreased by 75.4%


Make the final dataset

In [7]:
X=pd.concat([trn,tst],0)
del([trn,tst])
gc.collect()

33

Scale the data

In [8]:
from sklearn.preprocessing import StandardScaler
ls=list(X)
ss=StandardScaler()
X=pd.DataFrame(ss.fit_transform(X))
X.columns=ls
X=reduce_mem_usage(X)

Memory usage of dataframe is 4168.86 MB
Memory usage after optimization is: 1042.22 MB
Decreased by 75.0%


Create warmup lr

In [9]:

def custom_gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as K


class WarmUpLearningRateScheduler(keras.callbacks.Callback):
    """Warmup learning rate scheduler
    """

    def __init__(self, warmup_batches, init_lr, verbose=0):
        """Constructor for warmup learning rate scheduler

        Arguments:
            warmup_batches {int} -- Number of batch for warmup.
            init_lr {float} -- Learning rate after warmup.

        Keyword Arguments:
            verbose {int} -- 0: quiet, 1: update messages. (default: {0})
        """

        super(WarmUpLearningRateScheduler, self).__init__()
        self.warmup_batches = warmup_batches
        self.init_lr = init_lr
        self.verbose = verbose
        self.batch_count = 0
        self.learning_rates = []

    def on_batch_end(self, batch, logs=None):
        self.batch_count = self.batch_count + 1
        lr = K.get_value(self.model.optimizer.lr)
        self.learning_rates.append(lr)

    def on_batch_begin(self, batch, logs=None):
        if self.batch_count <= self.warmup_batches:
            lr = self.batch_count*self.init_lr/self.warmup_batches
            K.set_value(self.model.optimizer.lr, lr)
            if self.verbose > 0:
                print('\nBatch %05d: WarmUpLearningRateScheduler setting learning '
                      'rate to %s.' % (self.batch_count + 1, lr))
warm_up_lr = WarmUpLearningRateScheduler(400, init_lr=0.005)

Create data generator

In [10]:
from keras.utils import Sequence
class DAESequence(Sequence):
    def __init__(self,df,no_dum,frac=0.15,dumm=range(911),batch_size=2048):
        self.batch_size=batch_size
        self.frac=0.15
        self.dumm=dumm
        self.df=df
        self.cat_data=df[dumm].values
        self.num_data=df[no_dum].values
        self.no_dumm=no_dum
        self.len_data=df.shape[0]
        self.columns=df.shape[1]
        self.data=df
        self.idx=[]
        
        
        
    def __len__(self):
        return int(ceil(self.len_data/self.batch_size))
    
    
    
    def __getitem__(self,idx):
        self.idx.append(idx)
        last=min((idx+1)*self.batch_size,self.len_data)
        idx=idx*self.batch_size
        size=last-idx
        
        
        inps=[]
        outs=[]
        output_x=self.data.iloc[idx:last]
        
        
        
        
        data=output_x[self.no_dumm].values
        noise_x=data.copy()
        for i in range(len(self.no_dumm)):
            to=np.random.randint(0,size,int(size*self.frac))
            frm=np.random.randint(0,size,int(size*self.frac))
            noise_x[to,i]=noise_x[frm,i]
            
        inps.append(noise_x)
        outs.append(data)
        
        data=output_x[self.dumm].values
        noise_x=data.copy()
        for i in range(len(self.dumm)):
            to=np.random.randint(0,size,int(size*self.frac))
            frm=np.random.randint(0,size,int(size*self.frac))
            noise_x[to,i]=noise_x[frm,i]
        
        
        
        inps.append(noise_x)
        outs.append(data)
        
        
        
        
        
        return inps,outs

Fill the std columns and drop columns having nan mean

In [11]:
from tqdm import tqdm
a=X.isna().sum()
a=a[a>0]
cls=list(X)
for col in tqdm(list(a.index)):
  if col in cls:
    X[col]=X[col].fillna(X[col].mean())
a=X.isna().sum()
a=a[a>0]
for col in tqdm(list(a.index)):
  X=X.drop([col],1)

0it [00:00, ?it/s]
0it [00:00, ?it/s]


Create model

In [12]:
from keras.layers import concatenate
from keras.optimizers import Adam
def create_model():
    K.clear_session()
    num_inp = Input(shape=(num_shape,))
    cat_inp = Input(shape=(cat_shape,))
    inps = concatenate([num_inp, cat_inp])
    x = Dense(512, activation=custom_gelu)(inps)
    a = Dense(256, activation=custom_gelu)(x)
    b = Dense(512, activation = custom_gelu)(a)
    x = Dropout(.2)(b)
    cat_out = Dense(cat_shape, activation = "linear")(x)
    num_out = Dense(num_shape, activation = "linear")(x)
    model = Model(inputs=[num_inp,cat_inp], outputs=[num_out, cat_out])
    model.compile(
        optimizer=Adam(.05, clipnorm = 1, clipvalue = 1),
        loss=["mse", "mse"]
    )
    return model

Fitting the model

In [13]:
X=X.reset_index(drop=True)
num=[ i for i in list(X) if i not in tot]
batch_size=2048
auto_ckpt = ModelCheckpoint("ae.model", monitor='loss', verbose=1, save_best_only=True, save_weights_only=True, mode='min', period=1)
warm_up_lr = WarmUpLearningRateScheduler(400, init_lr=0.0001)
num_shape=len(num)
cat_shape=len(tot)
model_mse = create_model()
from math import *
gc.collect()
epochs = 1000
batch_size=2048
train_gen=DAESequence(X,num,batch_size=batch_size,dumm=tot)
hist = model_mse.fit_generator(train_gen, steps_per_epoch=len(X)//batch_size, epochs=epochs,
                           verbose=1, workers=-1,
                           use_multiprocessing=True,
                              callbacks=[auto_ckpt, warm_up_lr])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/1000
534/535 [============================>.] - ETA: 0s - loss: 1.6131 - dense_4_loss: 0.9947 - dense_3_loss: 0.6185
Epoch 00001: loss improved from inf to 1.61281, saving model to ae.model
535/535 [==============================] - 23s 43ms/step - loss: 1.6128 - dense_4_loss: 0.9949 - dense_3_loss: 0.6179
Epoch 2/1000
534/535 [============================>.] - ETA: 0s - loss: 1.0734 - dense_4_loss: 0.8759 - dense_3_loss: 0.1976
Epoch 00002: loss improved from 1.61281 to 1.07314, saving model to ae.model
535/535 [==============================] - 23s 42ms/step - loss: 1.0731 - dense_4_loss: 0.8757 - dense_3_loss: 0.1975
Epoch 3/1000
534/535 [============================>.] - ETA: 0s - loss: 0.9417 - dense_4_loss: 0.7807 - dense_3_loss: 0.1609
Epoch 00003: loss improved from 1.07314 to 0.94182, saving model to ae.model
535/535 [==============================] - 23s 42ms/step - loss: 0.9418 - dense_4_loss

KeyboardInterrupt: ignored

In [ ]:
mod=Model(inputs=model_mse.inputs,outputs=model_mse.layers[4].output)
mod.summary()

In [ ]:
pre=mod.predict((X[num],X[tot]))
pre.shape

In [ ]:
np.save('/content/gdrive/My Drive/fraud/without_id_withpoutV.npy',pre)